In [3]:
import os
import re
import numpy as np
import pytz
from datetime import datetime
from netCDF4 import Dataset
from scipy.spatial import cKDTree
from timezonefinder import TimezoneFinder

def get_variable(dataset, variable_names, potential_paths):
    """
    Attempts to retrieve a variable from multiple possible names and locations within a NetCDF dataset.
    Handles both root level and nested group structures.
    """
    # Check in the root group first
    for variable_name in variable_names:
        if variable_name in dataset.variables:
            return dataset.variables[variable_name][:]
    
    # Check in the specified paths if not found in the root
    for path in potential_paths:
        current_object = dataset
        for group_name in path.split('/'):
            if group_name and group_name in current_object.groups:
                current_object = current_object.groups[group_name]
            else:
                break
        else:
            for variable_name in variable_names:
                if variable_name in current_object.variables:
                    return current_object.variables[variable_name][:]
    
    raise KeyError(f"None of the variable names {variable_names} were found in any of the specified paths.")

def convert_utc_to_local(utc_dt, lat, lon):
    """Converts UTC datetime to local datetime based on latitude and longitude."""
    tf = TimezoneFinder()
    
    if lon > 180:
        lon -= 360
    
    tz_str = tf.timezone_at(lng=lon, lat=lat)
    if tz_str is None:
        return utc_dt
    
    local_tz = pytz.timezone(tz_str)
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return local_dt.replace(tzinfo=None)

def find_easternmost_lon_lat(satellite_filepath, bounding_box):
    """Find the easternmost longitude and corresponding latitude within the bounding box in the satellite file."""
    with Dataset(satellite_filepath, 'r') as satellite_data:
        print("Available groups in the dataset:", satellite_data.groups.keys())
        paths = ['', 'data_01', 'data_01/ku']  # Add potential paths

        # Print available variables for debugging
        for path in paths:
            current_object = satellite_data
            for group_name in path.split('/'):
                if group_name and group_name in current_object.groups:
                    current_object = current_object.groups[group_name]
                else:
                    break
            else:
                print(f"Available variables in path '{path}':", current_object.variables.keys())

        satellite_lons = get_variable(satellite_data, ['longitude', 'lon'], paths)
        satellite_lats = get_variable(satellite_data, ['latitude', 'lat'], paths)

        satellite_lons = np.where(satellite_lons > 180, satellite_lons - 360, satellite_lons)

        within_bbox = (
            (satellite_lats >= bounding_box[0]) &
            (satellite_lats <= bounding_box[1]) &
            (satellite_lons >= bounding_box[2]) &
            (satellite_lons <= bounding_box[3])
        )

        if np.any(within_bbox):
            easternmost_index = np.argmax(satellite_lons[within_bbox])
            easternmost_lon = satellite_lons[within_bbox][easternmost_index]
            easternmost_lat = satellite_lats[within_bbox][easternmost_index]
            return easternmost_lat, easternmost_lon
        else:
            print(f"No points within bounding box for file: {satellite_filepath}")
            return None, None

def process_satellite_file(filepath, adcirc_tree, bounding_box):
    """
    Processes a single satellite file to extract and compute necessary satellite data points.
    """
    try:
        satellite_data = Dataset(filepath, 'r')
        paths = ['', 'data_01', 'data_01/ku']
        iono_corr = get_variable(satellite_data, ['iono_corr_gim', 'iono_cor_alt', 'iono_corr'], paths)
        range_ocean_qual = get_variable(satellite_data, ['range_ocean_qual', 'qual_alt_1hz_range', 'range_ocean_compression_qual'], paths)
        ssb_corr = get_variable(satellite_data, ['sea_state_bias'], paths)
        range_ocean = get_variable(satellite_data, ['range_ocean', 'range'], paths)
        geoid = get_variable(satellite_data, ['geoid'], paths)
        altitude = get_variable(satellite_data, ['altitude', 'alt'], paths)
        dry_tropo_corr = get_variable(satellite_data, ['model_dry_tropo_corr_meas_alt', 'model_dry_tropo_cor_measurement_altitude'], paths)
        wet_tropo_corr = get_variable(satellite_data, ['model_wet_tropo_corr_meas_alt', 'model_wet_tropo_cor_measurement_altitude'], paths)
        satellite_lats = get_variable(satellite_data, ['latitude', 'lat'], paths)
        satellite_lons = get_variable(satellite_data, ['longitude', 'lon'], paths)
        satellite_lons = np.where(satellite_lons > 180, satellite_lons - 360, satellite_lons)

        closest_nodes = {}
        for i in range(len(geoid)):
            if range_ocean_qual[i] != 0:
                continue
            sat_lon = satellite_lons[i]
            topo = altitude[i] - (range_ocean[i] + iono_corr[i] + ssb_corr[i] + dry_tropo_corr[i] + wet_tropo_corr[i] + geoid[i])
            
            if not (-5 < topo < 5) or np.isnan(topo):
                continue
            
            if bounding_box[0] <= satellite_lats[i] <= bounding_box[1] and bounding_box[2] <= sat_lon <= bounding_box[3]:
                distance, index = adcirc_tree.query([satellite_lats[i], sat_lon])
                node_number = index + 1
                if node_number not in closest_nodes or distance < closest_nodes[node_number]['distance']:
                    closest_nodes[node_number] = {'distance': distance, 'topo': topo}
        return closest_nodes
    except Exception as e:
        print(f"Failed to process {filepath}: {str(e)}")
        return {}
    finally:
        satellite_data.close()

def parse_filename_datetime(filename):
    """Parses the datetime from the filename using various regex patterns and their corresponding datetime formats."""
    patterns = [
        (r'(\d{8})T(\d{6})', '%Y%m%dT%H%M%S'),
        (r'(\d{8})_(\d{6})', '%Y%m%d_%H%M%S'),
        (r'(\d{8})-(\d{6})', '%Y%m%d-%H%M%S'),
        (r'\d{8}T\d{6}',     '%Y%m%dT%H%M%S'),
    ]
    for pattern, fmt in patterns:
        match = re.search(pattern, filename)
        if match:
            return datetime.strptime(match.group(), fmt)
    return None

def main():
    satellite_directory = '/work2/07174/soelem/data/swotnad'
    adcirc_data_path = '/work2/07174/soelem/hopper/fort.63.nc'
    output_file_path = 'swot_flag.dat'
    bounding_box = [7, 60, -100, -58]
    start_time = datetime(2023, 8, 1)

    adcirc_data = Dataset(adcirc_data_path, 'r')
    adcirc_lats = adcirc_data.variables['y'][:]
    adcirc_lons = np.where(adcirc_data.variables['x'][:] > 180, adcirc_data.variables['x'][:] - 360, adcirc_data.variables['x'][:])
    adcirc_tree = cKDTree(np.column_stack((adcirc_lats, adcirc_lons)))
    adcirc_data.close()

    files_datetimes = [(f, parse_filename_datetime(f)) for f in os.listdir(satellite_directory)]
    files_datetimes = [fd for fd in files_datetimes if fd[1] is not None]
    files_datetimes.sort(key=lambda x: x[1])

    with open(output_file_path, 'w') as file:
        file.write("# SWOT Flag observations\n3600.0\n0.0\n")
        last_written_hour = 0

        for i, (filename, file_datetime) in enumerate(files_datetimes):
            filepath = os.path.join(satellite_directory, filename)
            
            if i == 0:
                local_time = convert_utc_to_local(file_datetime, bounding_box[0], bounding_box[2])
                current_hour = int((local_time - start_time).total_seconds() / 3600)
                if current_hour > 0:
                    file.write("##\n" * current_hour)
                last_written_hour = current_hour
            else:
                local_time = convert_utc_to_local(file_datetime, bounding_box[0], bounding_box[2])
                current_hour = int((local_time - start_time).total_seconds() / 3600)

            if current_hour > last_written_hour:
                file.write("##\n" * (current_hour - last_written_hour))
                last_written_hour = current_hour

            easternmost_lat, easternmost_lon = find_easternmost_lon_lat(filepath, bounding_box)
            if easternmost_lat is not None and easternmost_lon is not None:
                closest_nodes = process_satellite_file(filepath, adcirc_tree, bounding_box)
                for node, info in closest_nodes.items():
                    topo_rounded = round(info['topo'], 4)
                    file.write(f"{node} {topo_rounded}\n")

if __name__ == "__main__":
    main()

Available groups in the dataset: dict_keys(['data_01', 'data_20'])
Available variables in path 'data_01': dict_keys(['time', 'time_tai', 'index_first_20hz_measurement', 'numtotal_20hz_measurement', 'latitude', 'longitude', 'rad_side_1_surface_type_flag', 'rad_side_2_surface_type_flag', 'rad_side_1_distance_to_land', 'rad_side_2_distance_to_land', 'surface_classification_flag', 'angle_of_approach_to_coast', 'distance_to_coast', 'rad_tb_187_qual', 'rad_tb_238_qual', 'rad_tb_340_qual', 'rad_side_1_land_frac_187', 'rad_side_2_land_frac_187', 'rad_side_1_land_frac_238', 'rad_side_2_land_frac_238', 'rad_side_1_land_frac_340', 'rad_side_2_land_frac_340', 'alt_state_band_seq_flag', 'orb_state_diode_flag', 'orb_state_rest_flag', 'meteo_map_availability_flag', 'wave_model_map_availability_flag', 'sig0_cor_atm_source', 'rain_flag', 'rad_side_1_rain_flag', 'rad_side_2_rain_flag', 'ice_flag', 'rad_side_1_sea_ice_flag', 'rad_side_2_sea_ice_flag', 'rad_wet_tropo_cor_interp_qual', 'mean_sea_surface_cn